# Laboratorio 1 - definitions similarity

Misurazione dell’overlap lessicale tra una serie di definizioni per concetti generici/specifici concreti/astratti.
Partendo dai dati sulle definizioni (presente nella cartella "dati" su Moodle), si richiede di calcolare la similarità 2-a-2 tra le definizioni (ad es. usando la cardinalità dell'intersezione dei lemmi normalizzata sulla lunghezza minima delle definizioni), aggregando (ed effettuando la media degli score di similarità) sulle due dimensioni (concretezza / specificità).

## Import delle librerie

In [1]:
base_folder = './data'
!pip install sentence_transformers
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import itertools
from statistics import mean
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm

[nltk_data] Downloading package stopwords to /Users/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mario/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Lettura del file in input

Dal file *tsv* estraggo le definizioni associate ai termini. Per praticità, le definizioni sono organizzate in un dizionario la cui chiave è il termine.

In [2]:
def read_data(path):
    with open(path) as f:
        lines = f.readlines()
    lines = [x.strip().split('\t') for x in lines]

    defs = {}
    for i, e in enumerate(lines[0][1:]):
        defs[e] = [line[i+1].strip() for line in lines[1:]]
    return defs

In [3]:
definitions = read_data(f'{base_folder}/TLN-definitions-23.tsv')
display(definitions)

{'door': ['A construction used to divide two rooms, temporarily closing the passage between them',
  "It's an opening, it can be opened or closed.",
  'An object that divide two room, closing an hole in a wall. You can open the door to let people enter or get out.',
  'Usable for access from one area to another',
  'Structure that delimits an area and allows access to it',
  'an object that is used to block passage but can be moved to pass',
  'An assembled object, historically made of wood, but also of iron or other materials, used to separate rooms in a building. Sometimes opened by moving a handle, or pushed, or locked and requires some means to unlock. it consists of the main body, the hinges on which it rotates, and a lock.',
  'object used to go through rooms separate by a wall, can be opened or closed',
  'something that can be opened, in order to access to another place',
  'the access to a room',
  'an object that allows access to a room',
  'Enclosing of an entrance that bloc

## Similarities

Il metodo *print_table* permette di stampare i risultati in modo formattato.

A seguire, una serie di raffinamenti successivi per valutare l'efficacia delle modifiche apportate alle definizioni.

In [4]:
def print_table(sims):
    # door, ladybug, pain, blurriness
    print(f'{"":10s}{"Abstract":25s}{"Concrete":25s}')
    print(f'{"Generic":10s}{str(sims[2]):25s}{str(sims[0]):25s}')
    print(f'{"Specific":10s}{str(sims[3]):25s}{str(sims[1]):25s}')

### Similarità naive

Come primo test, calcolo l'overlap lessicale senza nessun tipo di pre-processing. Si noti che tale sistema restituirà stime "a rialzo", in quanto tiene conto delle stopwords che non descrivono nessun termine.

In [5]:
similarities = {k: [] for k in definitions}
for k, v in definitions.items():
    for def1, def2 in itertools.combinations(v, 2):
        def1, def2 = set(def1.split()), set(def2.split())
        similarities[k].append(len(def1.intersection(def2)) / min(len(def1), len(def2)))
similarities = [(k, round(mean(v)*100, 2)) for k, v in similarities.items()]
print_table(similarities)

          Abstract                 Concrete                 
Generic   ('pain', 19.96)          ('door', 26.89)          
Specific  ('blurriness', 18.08)    ('ladybug', 39.25)       


### Similarità case-insensitive

Una prima modifica è valutare le similarità con un confronto uniforme lower-case.

Come ragionevole aspettarsi, i risultati migliorano.

In [6]:
similarities = {k: [] for k in definitions}
for k, v in definitions.items():
    for def1, def2 in itertools.combinations(v, 2):
        def1, def2 = set(def1.lower().split()), set(def2.lower().split())
        similarities[k].append(len(def1.intersection(def2)) / min(len(def1), len(def2)))
similarities = [(k, round(mean(v)*100, 2)) for k, v in similarities.items()]
print_table(similarities)

          Abstract                 Concrete                 
Generic   ('pain', 23.65)          ('door', 29.46)          
Specific  ('blurriness', 20.72)    ('ladybug', 45.33)       


### Similarità senza stopwords

Procedo ora con un'operazione di pre-processing delle definizioni, rimuovendo sia la punteggiatura che le stopwords.

In questo caso, si nota un comportamento diverso in base ai termini.

Da una parte si può notare come il termine *ladybug* aumenti la similarità in quanto risulta più semplice per gli annotatori essere concordi sulle definizioni. Al contrario, *blurriness* registra un crollo del punteggio di similarità. Questo è dovuto al fatto che trovare una definizione appropriata risulta più complicato e quindi il precedente punteggio era caratterizzato in larga parte dalla sovrapposizione delle stopwords.

In [7]:
def remove_stopwords(phrase):
    punct = string.punctuation
    for p in punct:
        phrase = {item.replace(p, '') for item in phrase}
    phrase = {item.replace('\'s', '') for item in phrase}
    stop = stopwords.words('english')
    return {t for t in phrase if t not in stop}

In [8]:
similarities = {k: [] for k in definitions}
for k, v in definitions.items():
    for def1, def2 in itertools.combinations(v, 2):
        def1, def2 = remove_stopwords(def1.lower().split()), remove_stopwords(def2.lower().split())
        similarities[k].append(len(def1.intersection(def2)) / min(len(def1), len(def2)))
similarities = [(k, round(mean(v)*100, 2)) for k, v in similarities.items()]
print_table(similarities)

          Abstract                 Concrete                 
Generic   ('pain', 19.79)          ('door', 16.76)          
Specific  ('blurriness', 6.3)      ('ladybug', 55.6)        


### Similarità con Lemmatization

Oltre alla rimozione delle stopwords, le parole content possono essere uniformate tramite un processo di lemmatizzazione. Tale processo permette di ricavare la radice dei termini, evitando mismatch dovuti a variazioni morfologiche.

Per questo motivo, i punteggi aumentano leggermente rispetto alla semplice rimozione delle stopwords.

In [9]:
lemmatizer = WordNetLemmatizer()
similarities = {k: [] for k in definitions}
for k, v in definitions.items():
    for def1, def2 in itertools.combinations(v, 2):
        def1, def2 = remove_stopwords(def1.lower().split()), remove_stopwords(def2.lower().split())
        d1 = {lemmatizer.lemmatize(p) for p in def1}
        d2 = {lemmatizer.lemmatize(p) for p in def2}
        similarities[k].append(len(d1.intersection(d2)) / min(len(d1), len(d2)))
similarities = [(k, round(mean(v)*100, 2)) for k, v in similarities.items()]
print_table(similarities)

          Abstract                 Concrete                 
Generic   ('pain', 20.05)          ('door', 19.36)          
Specific  ('blurriness', 7.14)     ('ladybug', 55.93)       


### Similarità con Sentence-Bert

Infine, si può utilizzare il metodo di encoding Sentence-Bert per rappresentare le definizioni in formato vettoriale. Le definizioni passate in input al modello non hanno nessun tipo di pre-processing e, in questo caso, non si misura l'overlap lessicale ma la cosine-similarity tra i due vettori.

Questo metodo ottiene i risultati migliori, dimostrando come gli embedding prodotti tramite il trasformer siano in grado di catturare la vicinanza semantica in modo migliore rispetto al semplice overlap lessicale.

In [10]:
model = SentenceTransformer('all-mpnet-base-v2')
similarities = {k: [] for k in definitions}
for k, v in definitions.items():
    embeddings = [model.encode(def_i) for def_i in v]
    for def1, def2 in itertools.combinations(embeddings, 2):
        similarities[k].append(np.dot(def1, def2)/(norm(def1)*norm(def2)))
similarities = [(k, round(mean(v)*100, 2)) for k, v in similarities.items()]
print_table(similarities)

          Abstract                 Concrete                 
Generic   ('pain', 58.56)          ('door', 50.74)          
Specific  ('blurriness', 44.17)    ('ladybug', 67.56)       
